Комментарии:

ВВП как правило оценивают в темпах роста. При этом используется ВВП в ценах базового года. 
В нашем же датасете представлены данные ВВП в ценах текущего года.
Поэтому я взяла ВВП в локальной валюте на 2010 год.

На вкладке "Electric power consumption" много ошибок, выгрузила заново (надеюсь это честно :)).

Почистила справочник с регионами (добавила отсутствующие страны и взяла более общие названия регионов)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime, time
from datetime import timedelta
from plotly import offline
import pycountry
from pytz import country_timezones
from hdx.location.country import Country
import ccy

from gspread_pandas import Spread, Client
client = Client()

In [ ]:
df = pd.read_excel('Data Analyst Task.xlsx', sheet_name=None)

In [ ]:
df.keys()

In [ ]:
df_regions = df['Regions'][['Name', 'Region']].rename(columns={'Name': 'country_name'})
df_regions = df_regions[~df_regions.country_name.isna()]

# Удаляем Tibet и Great Britain. Так как по сути таких стран нет и в данных по ним нет информации 
# Также в справочике есть Netherlands Antilles, такой страны на данный момент не существует. 
# Но в данных фигурируют Кюрасао, Saint Martin (немецкая и французская части). 
# Эти страны ранее входили в состав Netherlands Antilles. Удалаяем Netherlands Antilles и добавляем вручную коды существующих стран.

df_regions = df_regions[~df_regions.country_name.isin(['Antarctica', 
                                                       'EnglishEnglish Name', 
                                                       'Great Britain', 
                                                       'Netherlands Antilles',
                                                       'Tibet'
                                                      ])]

# Определяем ISO 3 код страны по названию, чтобы избежать ошибок в названиях при мердже
df_regions['iso3'] = df_regions.country_name.apply(lambda x: Country.get_iso3_country_code_fuzzy(x)[0])
# У Косово нет ISO кода в справочнике, присваиваем вручную
df_regions.loc[df_regions.country_name == 'Kosovo', 'iso3'] = 'XKX'
# Некорректно определяется ISO код для одной из стран с названием Конго, меняем вручную
df_regions.loc[df_regions.country_name == 'Congo, Dem. Rep.', 'iso3'] = 'COD'

# Есть дубли 
#    East Timor (Timor-Leste) и Timor-Leste (East Timor)
#    Holy See и Vatican City State (Holy See)

df_regions = df_regions[['iso3']].drop_duplicates()
# Добавляем вручную страны, которые есть в данных, но нет в справочнике по регионам
new_iso3 = pd.DataFrame(['CUW', 'IMN', 'SXM', 'MAF'], columns=['iso3'])
df_regions = pd.concat([df_regions, new_iso3], sort=False).reset_index(drop=True)

def get_region(iso3, param):
    # Достаем информацию по стране с помощью библиотеки hdx (в дальнейшем отсюда возьмем регионы)
    country_info = Country.get_country_info_from_iso3(iso3)
    if pd.isna(country_info):
        #print(iso3)
        return np.NaN
    else:
        return country_info[param]

df_regions['region_name'] = df_regions.iso3.apply(lambda x: get_region(x, '#region+name+preferred+sub'))
df_regions['region_main'] = df_regions.iso3.apply(lambda x: get_region(x, '#region+main+name+preferred'))
df_regions['iso2'] = df_regions.iso3.apply(lambda x: get_region(x, '#country+code+v_iso2'))

df_regions.loc[df_regions.iso3 == 'XKX', 'region_name'] = 'Southern Europe'
df_regions.loc[df_regions.iso3 == 'XKX', 'region_main'] = 'Europe'
df_regions.loc[df_regions.iso3 == 'XKX', 'iso2'] = 'RS'
df_regions.loc[df_regions.iso3 == 'COD', 'iso2'] = 'CD'

df_regions['region'] = df_regions.region_main
df_regions.loc[df_regions.region_main.isin(['Africa']), 'region'] = 'Африка'
df_regions.loc[df_regions.region_main.isin(['Europe']), 'region'] = 'Европа'
df_regions.loc[df_regions.region_main.isin(['Oceania']), 'region'] = 'Океания'
df_regions.loc[df_regions.region_main.isin(['Asia']), 'region'] = 'Южная и Восточная Азия'
df_regions.loc[df_regions.region_name.isin(['Northern America']), 'region'] = 'Северная Америка'
df_regions.loc[df_regions.region_name.isin(['Latin America and the Caribbean']), 'region'] = 'Латинская Америка и Карибы'
df_regions.loc[(df_regions.region_name.isin(['Western Asia', 'Central Asia'])) | (df_regions.iso3 == 'RUS'), 'region'] = 'Россия и Центральная Азия'



In [ ]:
for sheet in df.keys():
    
    if (sheet != 'Task') and (sheet != 'Regions'):
        
        df[sheet] = df[sheet].iloc[:,0:6]
        df[sheet] = df[sheet].rename(columns=lambda x: str(x).strip())
        df[sheet] = df[sheet].rename(columns={'Country name': 'country_name'})
        df[sheet].country_name = df[sheet].country_name.str.strip()
        df[sheet] = df[sheet].set_index('country_name').replace(to_replace=r',', value='', regex=True).astype(float).reset_index()
        df[sheet]['iso3'] = df[sheet].country_name.apply(lambda x: Country.get_iso3_country_code_fuzzy(x)[0])
        df[sheet].loc[df[sheet].country_name == 'Kosovo', 'iso3'] = 'XKX'
        df[sheet].loc[df[sheet].country_name == 'Congo, Dem. Rep.', 'iso3'] = 'COD'




In [ ]:
columns_years = ['2005', '2006', '2007', '2008', '2009']
df_gdp_changes = df['GDP_LC'][(~df['GDP_LC']['2005'].isna()) & (~df['GDP_LC']['2006'].isna()) & (~df['GDP_LC']['2007'].isna()) & (~df['GDP_LC']['2008'].isna()) & (~df['GDP_LC']['2009'].isna()) & (~df['GDP_LC']['iso3'].isna())]

df_gdp_changes = pd.melt(df_gdp_changes.reset_index(), id_vars=['country_name', 'iso3'], value_vars=columns_years, var_name='year', value_name='gdp_lc').sort_values(by='year')
df_gdp_changes = df_gdp_changes.merge(df_regions, on='iso3', how='left')

df_ = pd.melt(df['Population, total '], id_vars=['iso3'], value_vars=columns_years, var_name='year', value_name='population')
df_gdp_changes = df_gdp_changes.merge(df_, on=['iso3', 'year'], how='inner')

df_ = pd.melt(df['GDP_USD_2010'], id_vars=['iso3'], value_vars=columns_years, var_name='year', value_name='gdp_usd_2010')
df_gdp_changes = df_gdp_changes.merge(df_, on=['iso3', 'year'], how='left')

# Рассчитываю изменения в ВВП, после чего нормирую значения на объем ВВП для каждой страны.

df_gdp_changes['gdp_changes_prev'] = df_gdp_changes.groupby(['iso3']).gdp_lc.shift()
df_gdp_changes['gdp_changes'] = (df_gdp_changes.gdp_lc - df_gdp_changes.gdp_changes_prev  ) /df_gdp_changes.gdp_changes_prev
df_gdp_changes.gdp_usd_2010.fillna(0, inplace=True)
df_gdp_changes['gdp_coeff'] = df_gdp_changes.groupby(['year']).gdp_usd_2010.transform(lambda x: x/sum(x))
df_gdp_changes['gdp_weighted_changes'] = df_gdp_changes.gdp_changes * df_gdp_changes.gdp_coeff * 100


df_ = pd.melt(df['Energy imports'], id_vars=['iso3'], value_vars=columns_years, var_name='year', value_name='energy_imports')
df_gdp_changes = df_gdp_changes.merge(df_, on=['iso3', 'year'], how='left')
df_gdp_changes['energy_export_mean'] = df_gdp_changes.groupby(['iso3']).energy_imports.transform('mean')
df_gdp_changes['is_energy_export'] = False
df_gdp_changes.loc[df_gdp_changes.energy_export_mean < 0, 'is_energy_export'] = True


df_ = pd.melt(df['Agriculture, value added'], id_vars=['iso3'], value_vars=columns_years, var_name='year', value_name='agriculture')
df_gdp_changes = df_gdp_changes.merge(df_, on=['iso3', 'year'], how='left')
df_gdp_changes['agriculture_mean'] = df_gdp_changes.groupby(['iso3']).agriculture.transform('mean')
df_gdp_changes['is_agriculture'] = False
df_gdp_changes.loc[df_gdp_changes.agriculture_mean > 15, 'is_agriculture'] = True

# Рассчитываю изменения в потреблении электроэнергии, после чего нормирую значения на популяцию для каждой страны.

df_ = pd.melt(df['EPC_new'], id_vars=['iso3'], value_vars=columns_years, var_name='year', value_name='electric_power_consumption')
df_gdp_changes = df_gdp_changes.merge(df_, on=['iso3', 'year'], how='left')
df_gdp_changes['epc_prev'] = df_gdp_changes.groupby(['iso3']).electric_power_consumption.shift()
df_gdp_changes['epc_changes'] = (df_gdp_changes.electric_power_consumption - df_gdp_changes.epc_prev  ) /df_gdp_changes.epc_prev
df_gdp_changes['population_coeff'] = df_gdp_changes.groupby(['year']).population.transform(lambda x: x/sum(x))
df_gdp_changes['epc_weighted_changes'] = df_gdp_changes.epc_changes * df_gdp_changes.population_coeff * 100


In [ ]:
# Загружаю агрегированные данные в Google Sheet 'Последствия кризиса 2008'

spread = Spread('Последствия кризиса 2008')
df_ = df_gdp_changes[df_gdp_changes.year != '2005'].groupby(['year', 'region']).agg(gdp_weighted_changes=('gdp_weighted_changes', 'sum'), gdp_usd_2010=('gdp_usd_2010', 'sum')).reset_index()
spread.df_to_sheet(df_, index=False, sheet='Рост ВВП по регионам', replace=True)

df_ = df_gdp_changes[(df_gdp_changes.region == 'Африка')]
df_['gdp_coeff'] = df_.groupby(['year']).gdp_usd_2010.transform(lambda x: x/sum(x))
df_['gdp_weighted_changes'] = df_.gdp_changes * df_.gdp_coeff * 100
df_ = df_[(df_.year != '2005')].groupby(['year', 'is_energy_export']).gdp_weighted_changes.sum().reset_index()
spread.df_to_sheet(df_, index=False, sheet='Африка', replace=True)

df_ = df_gdp_changes[(df_gdp_changes.country_name.isin(df_gdp_changes[(df_gdp_changes.year == '2007') & (df_gdp_changes.is_agriculture == True)].sort_values(by='agriculture_mean', ascending=False).head(20).country_name)) & (df_gdp_changes.year != '2005')].sort_values(by='gdp_usd_2010')[['gdp_changes', 'year', 'country_name', 'gdp_usd_2010']]
spread.df_to_sheet(df_, index=False, sheet='Аграрные страны', replace=True)

df_ = df_gdp_changes[df_gdp_changes.year != '2005'].groupby(['year', 'region']).epc_weighted_changes.sum().reset_index()
spread.df_to_sheet(df_, index=False, sheet='Потребление электроэнергии Все', replace=True)


df_ = df_gdp_changes[(df_gdp_changes.region == 'Южная и Восточная Азия')]
df_['asia_region'] = 'Другие страны Азии'
df_.loc[df_.country_name.isin(['Vietnam']), 'asia_region'] = 'Вьетнам'
df_.loc[df_.country_name.isin(['Malaysia', 'Philippines', 'Indonesia', 'Thailand']), 'asia_region'] = 'Азиатские тигры *'
df_.loc[df_.country_name.isin(['China']), 'asia_region'] = 'Китай'
df_.loc[df_.country_name.isin(['Japan']), 'asia_region'] = 'Япония'
df_.loc[df_.country_name.isin(['India']), 'asia_region'] = 'Индия'
df_.loc[df_.country_name.isin(['Bangladesh', 'Myanmar']), 'asia_region'] = 'Бангладеш, Мьянма'
df_.loc[df_.iso3.isin(['IRN', 'AFG', 'PAK']), 'asia_region'] = 'Иран, Афганистан, Пакистан'
df_['population_coeff'] = df_.groupby(['year']).population.transform(lambda x: x/sum(x))
df_['epc_part'] = df_.electric_power_consumption * df_.population_coeff
df_['epc_weighted_changes'] = df_.epc_changes * df_.population_coeff * 100
df_ = df_[df_.year != '2005'].groupby(['year', 'asia_region']).agg(epc_part=('epc_part', 'sum'), epc_weighted_changes=('epc_weighted_changes', 'sum')).reset_index()
spread.df_to_sheet(df_, index=False, sheet='Потребление электроэнергии Азия', replace=True)



df_ = df_gdp_changes.copy(deep=True)
df_['is_asia'] = 'Остальной мир'
df_.loc[df_.region == 'Южная и Восточная Азия', 'is_asia'] = 'Южная и Восточная Азия'
df_ = df_.groupby(['year', 'is_asia']).agg(gdp_usd_2010=('gdp_usd_2010', 'sum')).reset_index()
spread.df_to_sheet(df_, index=False, sheet='Доля Азии в мировом ВВП', replace=True)
